# Debugging Practicum I
In this class, we'll use all the tools at our disposal to fix some code that isn't working. <a href="{{ site.dataurl }}/week4/pupil.mat">The data</a> for this exercise are time series of pupil measurements taken while a participant completed an ["oddball" task](https://en.wikipedia.org/wiki/Oddball_paradigm), and the analysis performed is plotting trial-averaged pupil responses to standard and deviant sounds, locked to stimulus presentation. As with most code in the wild, this is only lightly commented. This constitutes what is often called a [minimal working example](https://en.wikipedia.org/wiki/Minimal_Working_Example) (MWE), apart from the fact that it's not quite minimal and not quite working.

The complete code, put together as a Jupyter notebook, is available [here](https://colab.research.google.com/drive/1n8rwZZOX7gaHQ33XlH5EEVR5E6VAfVkP?usp=sharing), but is also listed below. Your goal in class, using whatever means you have at your disposal, is to find and remove the bugs.

```{admonition} Exercise
1. Fix the code. Note that not all bugs generate errors. To find some bugs, you will need to understand what the pieces of the code are trying to accomplish. Using the debugger to help you trace the code may be very helpful.
```



## Code listing:

In [ ]:
# download the data
!gdown 1t2JIa8WYFjHqm4Eg_pjSQQTZ44uH68ni

In [ ]:

dat = loadmat('data/pupil.mat', struct_as_record=False)

## Solution:

```{toggle}
1. In cell 4, `dat['pupil']` needs to be `squeeze`d.
1. In cell 8, the post-event time is wrong. It should be 8 to match the figure.
1. In cell 10, last line: The returned time axis should be a shifted `np.arange(nstart, nend + 1)`.
1. In cell 14, the `mean` and `std` should exclude `nan` (i.e., `nanmean`, `nanstd`).
1. In cell 15, line 3, the first `+` should be `-`.
```